In [ ]:
# v2 github: https://github.com/aws/aws-iot-device-sdk-python-v2
# install IoT SDK
!pip install awsiotsdk

In [1]:
from awscrt import io, mqtt, auth, http
from awsiot import mqtt_connection_builder
import sys
import threading
import time

In [2]:
import boto3

iot = boto3.client('iot')
endpoint = iot.describe_endpoint(endpointType='iot:Data-ATS')['endpointAddress']
endpoint

'a2xqi9tklutvnu-ats.iot.us-west-2.amazonaws.com'

In [ ]:
!ls ./iot-beyoung-01/

In [3]:
device = 'iot-beyoung-01'
cert = '{}/{}.cert.pem'.format(device, device)
pri_key = '{}/{}.private.key'.format(device, device)
root_ca = '{}/root-CA.crt'.format(device)
clientid = 'basicPubSub'

print('cert: {} \npri_key: {}\nroot_ca: {}'.format(cert, pri_key, root_ca))

cert: iot-beyoung-01/iot-beyoung-01.cert.pem 
pri_key: iot-beyoung-01/iot-beyoung-01.private.key
root_ca: iot-beyoung-01/root-CA.crt


In [4]:
# Spin up resources
event_loop_group = io.EventLoopGroup(1)
host_resolver = io.DefaultHostResolver(event_loop_group)
client_bootstrap = io.ClientBootstrap(event_loop_group, host_resolver)

In [5]:
# Callback when connection is accidentally lost.
def on_connection_interrupted(connection, error, **kwargs):
    print("Connection interrupted. error: {}".format(error))

# Callback when an interrupted connection is re-established.
def on_connection_resumed(connection, return_code, session_present, **kwargs):
    print("Connection resumed. return_code: {} session_present: {}".format(return_code, session_present))

    if return_code == mqtt.ConnectReturnCode.ACCEPTED and not session_present:
        print("Session did not persist. Resubscribing to existing topics...")
        resubscribe_future, _ = connection.resubscribe_existing_topics()

        # Cannot synchronously wait for resubscribe result because we're on the connection's event-loop thread,
        # evaluate result with a callback instead.
        resubscribe_future.add_done_callback(on_resubscribe_complete)    

In [ ]:
def on_resubscribe_complete(resubscribe_future):
        resubscribe_results = resubscribe_future.result()
        print("Resubscribe results: {}".format(resubscribe_results))

        for topic, qos in resubscribe_results['topics']:
            if qos is None:
                sys.exit("Server rejected resubscribe to topic: {}".format(topic))

In [6]:
mqtt_connection = mqtt_connection_builder.mtls_from_path(
    endpoint=endpoint,
    cert_filepath=cert,
    pri_key_filepath=pri_key,
    client_bootstrap=client_bootstrap,
    ca_filepath=root_ca,
    on_connection_interrupted=on_connection_interrupted,
    on_connection_resumed=on_connection_resumed,
    client_id=clientid,
    clean_session=False,
    keep_alive_secs=6)

In [7]:
connect_future = mqtt_connection.connect()

# Future.result() waits until a result is available
connect_future.result()
print("Connected!")

Connected!


In [13]:
import time, json

t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(current_time)

11:12:00


In [14]:
message = json.dumps("{} [{}]".format('hello from client', current_time))
topic = 'sdk/test/Python'
print("Publishing message to topic '{}': {}".format(topic, message))
mqtt_connection.publish(
    topic=topic,
    payload=message,
    qos=mqtt.QoS.AT_LEAST_ONCE)

Publishing message to topic 'sdk/test/Python': "hello from client [11:12:00]"


(<Future at 0x7f8bae6dc0f0 state=pending>, 1)